# WeatherPy

In [4]:
import matplotlib.pyplot as plt
import requests
import pandas as pd
import numpy as np
import random
from config import weather_api_key
from config import g_key
from citipy import citipy
from pprint import pprint


## Generate Cities List

In [15]:
lat = np.random.uniform(low = -90.000, high = 90.000, size = 100 )
lng = np.random.uniform(low = -180.000, high = 180.000, size = 100)
coordinates = zip(lat, lng)

city_list = []

for lat_long in coordinates:
    city = citipy.nearest_city(lat_long[0], lat_long[1])
    city = city.city_name
    
    if city not in city_list:
        city_list.append(city)    

# print(f'There are {len(city_list)} cities in the generated list.')
# city_list

### Perform API Calls

In [16]:
act_city_name = []
country = []
temperature = []
humidity = []
cloudiness = []
wind_speed = []
lattitude = []
longitude = []

temp_units = "imperial"
city_url = f"http://api.openweathermap.org/data/2.5/weather?appid={weather_api_key}&units={temp_units}&q="

record_counter = 0
set_counter = 1

for city_name in city_list:
    query = (city_url + city_name)
    response = requests.get(query).json()
    record_counter += 1
#     pprint(response)
    
    try:
        temperature.append(response['main']['temp_max'])
        humidity.append(response['main']['humidity'])
        cloudiness.append(response['clouds']['all'])
        wind_speed.append(response['wind']['speed'])
        lattitude.append(response['coord']['lat'])
        longitude.append(response['coord']['lat'])
        act_city_name.append(response['name'])
        country.append(response['sys']['country'])
        
        if record_counter > 50:
            record_counter = 1
            set_counter += 1
            
        print(f'Processing record {record_counter} of set {set_counter} | {city_name}.')
    
    except:
        record_counter -= 1
        print(f'City not found. Skipping...')
print('----------------------------------')
print('Data Retrieval Complete')
print('----------------------------------')


Processing record 1 of set 1 | hambantota.
Processing record 2 of set 1 | provideniya.
Processing record 3 of set 1 | bambous virieux.
Processing record 4 of set 1 | huntsville.
City not found. Skipping...
Processing record 5 of set 1 | dekar.
Processing record 6 of set 1 | beringovskiy.
Processing record 7 of set 1 | vao.
Processing record 8 of set 1 | narsaq.
City not found. Skipping...
Processing record 9 of set 1 | aden.
Processing record 10 of set 1 | panguna.
Processing record 11 of set 1 | yar-sale.
City not found. Skipping...
Processing record 12 of set 1 | saskylakh.
Processing record 13 of set 1 | belmonte.
Processing record 14 of set 1 | dikson.
Processing record 15 of set 1 | lebu.
Processing record 16 of set 1 | pacific grove.
City not found. Skipping...
Processing record 17 of set 1 | touros.
Processing record 18 of set 1 | abu samrah.
Processing record 19 of set 1 | port alfred.
Processing record 20 of set 1 | vila velha.
Processing record 21 of set 1 | cordoba.
Processi

## Convert Raw Data to DataFrame

In [21]:
# Create DataFrame
geo_weather_data_df = pd.DataFrame({
    'City' : act_city_name,
    'Lat' : lattitude,
    'Lng' : longitude,
    'Max Temp' : temperature,
    'Humidity' : humidity,
    'Cloudiness' : cloudiness,
    'Wind Speed' : wind_speed,
    'Country' : country,
})

# Add a column for hemisphere
def get_hemisphere(row):
    
    lattitude = row[2]    
    if lattitude >= 0:
        return "Northern Hemisphere"
    else:
        return "Southern Hemisphere"

# Call the get_hemisphere function to append a new Hemisphere column to the dataframe
geo_weather_data_df["Hemisphere"] = geo_weather_data_df.apply(get_hemisphere, axis="columns")

geo_weather_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hemisphere
0,Hambantota,6.12,6.12,87.80,70,40,11.41,LK,Northern Hemisphere
1,Provideniya,64.38,64.38,34.14,84,35,7.23,RU,Northern Hemisphere
2,Bambous Virieux,-20.34,-20.34,72.00,83,75,6.93,MU,Southern Hemisphere
3,Huntsville,34.73,34.73,64.00,100,20,3.67,US,Northern Hemisphere
4,Dekar,-21.53,-21.53,57.11,32,0,7.81,BW,Southern Hemisphere
